In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define constants
TOP_N = 10  # Number of top SP500 stocks to fetch
MOVING_AVERAGE_WINDOW = 9  # Simple Moving Average (SMA) window
ENTRY_THRESHOLD_PIPS = 0.001  # Entry threshold in percentage of price
EXIT_THRESHOLD_PIPS = 0.001  # Exit threshold in percentage of price

# Step 1: Fetch top 10 SP500 stocks and their data
def get_top_sp500_stocks():
    # In a real scenario, replace this with a proper S&P 500 top companies API or data source
    sp500_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'BRK-B', 'JPM', 'V']  # Example tickers
    return sp500_tickers[:TOP_N]

# Step 2: Download OHLCV data
def download_data(tickers):
    data = {}
    for ticker in tickers:
        print(f"Downloading data for {ticker}...")
        data[ticker] = yf.download(ticker, start="2020-01-01", end="2024-12-31")
    return data

# Step 3: Apply Moving Average and SVM logic
def apply_strategy(data, moving_average_window, entry_threshold_pips, exit_threshold_pips):
    results = {}
    for ticker, df in data.items():
        print(f"Processing {ticker}...")
        df['SMA'] = df['Close'].rolling(window=moving_average_window).mean()
        df['Entry Threshold'] = df['SMA'] * (1 + entry_threshold_pips)
        df['Exit Upper'] = df['SMA'] * (1 + exit_threshold_pips)
        df['Exit Lower'] = df['SMA'] * (1 - exit_threshold_pips)

        # Signal conditions
        df['Long Entry'] = (df['Close'] > df['Entry Threshold']) & (df['Close'].shift(1) <= df['Entry Threshold'])
        df['Short Entry'] = (df['Close'] < df['Entry Threshold']) & (df['Close'].shift(1) >= df['Entry Threshold'])
        df['Long Exit'] = (df['Close'] >= df['Exit Upper']) | (df['Close'] < df['Exit Lower'])
        df['Short Exit'] = (df['Close'] <= df['Exit Lower']) | (df['Close'] > df['Exit Upper'])

        # Simplistic P&L calculation (example)
        df['Position'] = 0
        df.loc[df['Long Entry'], 'Position'] = 1
        df.loc[df['Short Entry'], 'Position'] = -1
        df['Position'] = df['Position'].replace(to_replace=0, method='ffill')
        df['Daily Return'] = df['Position'] * df['Close'].pct_change()
        df['Cumulative Return'] = (1 + df['Daily Return']).cumprod()

        results[ticker] = df

    return results

# Step 4: Plot and analyze
def plot_results(results):
    for ticker, df in results.items():
        plt.figure(figsize=(12, 6))
        plt.plot(df.index, df['Close'], label='Close Price', alpha=0.7)
        plt.plot(df.index, df['SMA'], label='SMA', alpha=0.7)
        plt.fill_between(df.index, df['Exit Lower'], df['Exit Upper'], color='gray', alpha=0.2, label='Exit Band')
        plt.legend()
        plt.title(f"{ticker} - SVM Strategy")
        plt.show()

# Main script
def main():
    tickers = get_top_sp500_stocks()
    data = download_data(tickers)
    results = apply_strategy(data, MOVING_AVERAGE_WINDOW, ENTRY_THRESHOLD_PIPS, EXIT_THRESHOLD_PIPS)
    plot_results(results)

    # Performance summary
    for ticker, df in results.items():
        final_cum_return = df['Cumulative Return'].iloc[-1]
        print(f"{ticker} - Final Cumulative Return: {final_cum_return:.2f}")

if __name__ == "__main__":
    main()
